In [ ]:
#--------------------#
# Importing Packages #
#--------------------#
import yfinance as yf
import pandas as pd
import warnings
from datetime import datetime
from datetime import date, timedelta

#---------------------#
# Importing Variables #
#---------------------#
from variables.variables import DB_CONFIG, STOCKS, ETFS

In [ ]:
#---------------------#
# Importing Constants #
#---------------------#
from constants import COMPANY_INFO_COLUMNS, STOCK_FUNDAMENTALS_COLUMNS, STOCK_RECOMMENDATIONS_COLUMNS, PRICE_HISTORY_COLUMNS

HISTORY_START_DATE = date.today() + timedelta(days=-1)
HISTORY_END_DATE = date.today()

In [22]:
stock = yf.Ticker("KPN.AS")
#stock.calendar
stock.isin

'-'

In [4]:
def get_company_info(required_stocks, required_columns_dict):
    # Extract column names from the dictionary keys
    column_names = list(required_columns_dict.keys())
    Company_Info_df = pd.DataFrame(columns=column_names)

    for stock in required_stocks:
        temp_list = []
        company_info = yf.Ticker(stock).info

        for column in column_names:  # Iterate over the extracted column names
            if column in company_info:
                temp_list.append(company_info[column])
            else:
                temp_list.append(None)
                warnings.warn(f"Column '{column}' not found for stock '{stock}'")

        index_label = f"{stock} {datetime.now().replace(hour=0, minute=0, second=0, microsecond=0).strftime('%Y-%m-%d %H:%M:%S')}"
        Company_Info_df.loc[index_label] = temp_list

    return Company_Info_df

#get_company_info(STOCKS, COMPANY_INFO_COLUMNS)
#get_company_info(STOCKS, STOCK_FUNDAMENTALS_COLUMNS)
#get_company_info(STOCKS, STOCK_RECOMMENDATIONS_COLUMNS)

In [5]:
import mysql.connector

def store_data_in_mysql(df, table_name, column_definitions, db_config):
    # Connect to MySQL
    conn = mysql.connector.connect(**db_config)
    cursor = conn.cursor()

    # Create table query
    columns_with_types = ", ".join([f"{col} {dtype}" for col, dtype in column_definitions.items()])
    create_table_query = f"CREATE TABLE IF NOT EXISTS {table_name} ({columns_with_types})"
    cursor.execute(create_table_query)

    # Insert data into the table
    placeholders = ", ".join(["%s"] * len(column_definitions))  # Placeholder for values
    insert_query = f"INSERT INTO {table_name} ({', '.join(column_definitions.keys())}) VALUES ({placeholders})"

    for _, row in df.iterrows():
        cursor.execute(insert_query, tuple(row))

    # Commit changes and close the connection
    conn.commit()
    cursor.close()
    conn.close()

# Example usage
if __name__ == "__main__":
    
    TABLE_NAME = "company_info"

    # Get the company information
    df = get_company_info(STOCKS, COMPANY_INFO_COLUMNS)

    # Store the data in the MySQL table
    store_data_in_mysql(df, TABLE_NAME, COMPANY_INFO_COLUMNS, DB_CONFIG)

In [ ]:
def get_price_history(startDate, endDate, required_stocks, required_columns, interval="1h"): #period="2y"
    Price_History_df = pd.DataFrame(columns=required_columns)
    for stock in required_stocks:
        price_history = yf.Ticker(stock).history(start=startDate.strftime('%Y-%m-%d'), end=endDate.strftime('%Y-%m-%d'), interval=interval, actions=False)
        price_history['Symbol'] = stock
        Price_History_df = pd.concat([Price_History_df, price_history])

    return Price_History_df

get_price_history(HISTORY_START_DATE, HISTORY_END_DATE, STOCKS, PRICE_HISTORY_COLUMNS)
#get_price_history(ETFS, PRICE_HISTORY_COLUMNS)

KPN.AS: No price data found, symbol may be delisted (1h 2024-12-08 -> 2024-12-09)
SHELL.AS: No price data found, symbol may be delisted (1h 2024-12-08 -> 2024-12-09)


AD.AS: No price data found, symbol may be delisted (1h 2024-12-08 -> 2024-12-09)
PHIA.AS: No price data found, symbol may be delisted (1h 2024-12-08 -> 2024-12-09)
INGA.AS: No price data found, symbol may be delisted (1h 2024-12-08 -> 2024-12-09)
HEIJM.AS: No price data found, symbol may be delisted (1h 2024-12-08 -> 2024-12-09)


,Symbol,Open,High,Low,Close,Volume,Adj Close
